In [1]:
library(tidyverse)
library(lubridate)
library(ggplot2)
library(sf)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.2     v tibble    3.3.0
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.1.0     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'sf' was built under R version 4.5.2"
Linking to GEOS 3.13.1, GDAL 3.11.4, PROJ 9.7.0; sf_use_s2() is TRUE



In [2]:
# Load 2024 data
df_2024 <- read_csv("../data/offenses_known_csv_1960_2024_month/offenses_known_monthly_2024.csv")|>
    glimpse()

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 309168 Columns: 197
-- Column specification --------------------------------------------------------
Delimiter: ","
chr   (35): ori, ori9, agency_name, state, state_abb, month, last_month_repo...
dbl  (153): year, number_of_months_reported, month_missing, longitude, latit...
lgl    (8): number_of_months_missing, covered_by_population_group, populatio...
date   (1): date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 309,168
Columns: 197
$ ori                                     <chr> "AK00101", "AK00101", "AK00101~
$ ori9                                    <chr> "AK0010100", "AK0010100", "AK0~
$ agency_name                             <chr> "anchorage", "anchorage", "anc~
$ state                                   <chr> "alaska", "alaska", "alaska", ~
$ state_abb                               <chr> "AK", "AK", "AK", "AK", "AK", ~
$ year                                    <dbl> 2024, 2024, 2024, 2024, 2024, ~
$ month                                   <chr> "january", "february", "march"~
$ date                                    <date> 2024-01-01, 2024-02-01, 2024-~
$ number_of_months_missing                <lgl> NA, NA, NA, NA, NA, NA, NA, NA~
$ number_of_months_reported               <dbl> 12, 12, 12, 12, 12, 12, 12, 12~
$ last_month_reported                     <chr> "december", "december", "decem~
$ month_missing                           <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ fips_state_

In [3]:
# Identify all unique states in the dataset
unique_states <- df_2024 |> 
    distinct(state) |> 
    pull(state)

print(length(unique_states))

[1] 57


In [4]:
# identify all unique census regions in the 2024 data
unique_regions <- df_2024 |> 
    distinct(census_name) |> 
    pull(census_name)

# Print the unique census regions
print(length(unique_regions))

[1] 13070


In [5]:
# Trim the df
df_2024_trimmed <- df_2024 |>
    filter(!state_abb %in% c("AS", "GU", "MP", "PR", "VI"))|>
    filter(!is.na(census_name))|>
    select(state, state_abb, address_city, census_name, year, month, date, population_1, population, core_city_indication, 
            officers_killed_by_felony:actual_index_total)|>
    pivot_longer(cols = actual_murder:actual_assault_aggravated,
                 names_to = "offense_type",
                 values_to = "offense_count") |>
    mutate(crime_rate = offense_count/population_1 * 100000)|>
    glimpse()

Rows: 7,396,704
Columns: 19
$ state                       <chr> "alaska", "alaska", "alaska", "alaska", "a~
$ state_abb                   <chr> "AK", "AK", "AK", "AK", "AK", "AK", "AK", ~
$ address_city                <chr> "anchorage", "anchorage", "anchorage", "an~
$ census_name                 <chr> "anchorage municipality", "anchorage munic~
$ year                        <dbl> 2024, 2024, 2024, 2024, 2024, 2024, 2024, ~
$ month                       <chr> "january", "january", "january", "january"~
$ date                        <date> 2024-01-01, 2024-01-01, 2024-01-01, 2024-~
$ population_1                <dbl> 286958, 286958, 286958, 286958, 286958, 28~
$ population                  <dbl> 286958, 286958, 286958, 286958, 286958, 28~
$ core_city_indication        <chr> "core city of msa", "core city of msa", "c~
$ officers_killed_by_felony   <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ officers_killed_by_accident <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ officers_a

In [6]:
# Summarize crime count by type
crime_type_summary <- df_2024_trimmed|>
    group_by(offense_type)|>
    summarize(count = sum(offense_count),
            mean = mean(offense_count),
            sd = sd(offense_count))|>
    arrange(by = count)

crime_type_summary

offense_type,count,mean,sd
<chr>,<dbl>,<dbl>,<dbl>
actual_theft_under50_dollar,0,0.00000000,0.0000000
actual_manslaughter,1936,0.00732867,0.1127764
actual_rape_attempted,4253,0.01609960,0.2137941
actual_murder,15816,0.05987099,0.6773848
actual_robbery_with_a_knife,17347,0.06566655,1.7450008
actual_robbery_other_weapon,23760,0.08994276,1.5630838
actual_burglary_attempted,49420,0.18707792,2.1923817
actual_motor_vehicle_theft_other,61211,0.23171240,2.2356503
actual_robbery_with_a_gun,63611,0.24079752,3.6943929


In [ ]:
# Get the stat for major US cities
us